In [153]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score



In [154]:
df = pd.read_csv('Telco-Customer-Churn.csv')

In [155]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

cat_col = list(df.dtypes[df.dtypes == 'object'].index)

for col in cat_col: # loop through all the column in the strings varible
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [ ]:
df.totalcharges= pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)
df.churn= (df.churn == 'yes').astype(int)
df_full_train, df_test = train_test_split(df, test_size= 0.2, random_state=1)


In [160]:
numerical = ['tenure','monthlycharges', 'totalcharges' ]
categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod'
]

In [161]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
    
    model = LogisticRegression(C=C, max_iter=1000)
    
    model.fit(X_train, y_train) 
    
    return dv, model

In [162]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records') 
    
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1] 
    
    return y_pred

In [163]:
C = 1.0
n_splits = 5

In [164]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]
    
    y_train = df_train.churn.values
    y_val = df_val.churn.values
    
    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)
    
    
    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))    

c:\Users\Hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/m

C=1.0 0.842 +- 0.007


c:\Users\Hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [165]:
scores

[np.float64(0.8446279564490428),
 np.float64(0.8451633061846113),
 np.float64(0.8334256472004816),
 np.float64(0.8347728944170877),
 np.float64(0.8516323617081358)]

In [166]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model) 

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

c:\Users\Hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


np.float64(0.8583327916626041)

In [167]:
print("=" * 80)
print("SAVE A MODEL")
print("=" * 80)

SAVE A MODEL


In [ ]:
import pickle

In [169]:
output_file = f'model_C={C}.bin'
output_file

'model_C=1.0.bin'

In [170]:
with open(output_file, 'wb') as f_out:
    pickle.dumb((dv, model), f_out)
    



AttributeError: module 'pickle' has no attribute 'dumb'

In [ ]:
print("=" * 80)
print("LOAD THE MODEL")
print("=" * 80)

LOAD THE MODEL
